# Loading data from text file into Mongo DB

In [1]:
import pymongo

In [2]:
client = pymongo.MongoClient('localhost', 27017)

In [3]:
db = client['data_extract_db']

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'data_extract_db')

In [5]:
collection = db['income']

In [6]:
with open('income_header.txt') as f_in:
    column_headers = f_in.readline()

In [7]:
column_headers

'age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label'

In [8]:
cols = column_headers.split(',')
cols

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'label']

In [9]:
with open('income.txt') as f_in:
    for line in f_in:
        row_list = line.rstrip('\n').split(',')
        row_dict = dict(zip(cols, row_list))
        try:
            row_dict['age'] = int(row_dict['age'])            
            collection.insert_one(row_dict)
        except:
            pass

In [10]:
collection.count_documents(filter={})

65122

In [11]:
over_35 = collection.find({'age':{"$gt" : 35}})

In [12]:
over_35.next()

{'_id': ObjectId('5d21ef499d0776fb16b7ab85'),
 'age': 39,
 'workclass': ' State-gov',
 'fnlwgt': ' 77516',
 'education': ' Bachelors',
 'education-num': ' 13',
 'marital-status': ' Never-married',
 'occupation': ' Adm-clerical',
 'relationship': ' Not-in-family',
 'race': ' White',
 'sex': ' Male',
 'capital-gain': ' 2174',
 'capital-loss': ' 0',
 'hours-per-week': ' 40',
 'native-country': ' United-States',
 'label': ' <=50K'}

# Using index

In [13]:
import time

In [14]:
start = time.time()
age50 = collection.find({'age': {'$eq':50}})
end = time.time()
print('Time of find without indexing:', end - start)

Time of find without indexing: 0.0


In [15]:
index_result = db.profiles.create_index([('age', pymongo.ASCENDING)], unique=False)

In [16]:
start = time.time()
age45 = collection.find({'age': {'$eq':45}})
end = time.time()
print('Time of find with indexing:', end - start)

Time of find with indexing: 0.0


# Using dataframe fith Mongo DB

In [17]:
import pandas as pd

In [18]:
income_df = pd.DataFrame(list(collection.find()))

In [19]:
income_df.head()

,_id,age,capital-gain,capital-loss,education,education-num,fnlwgt,hours-per-week,label,marital-status,native-country,occupation,race,relationship,sex,workclass
0,5d21ef499d0776fb16b7ab85,39,2174,0,Bachelors,13,77516,40,<=50K,Never-married,United-States,Adm-clerical,White,Not-in-family,Male,State-gov
1,5d21ef499d0776fb16b7ab86,50,0,0,Bachelors,13,83311,13,<=50K,Married-civ-spouse,United-States,Exec-managerial,White,Husband,Male,Self-emp-not-inc
2,5d21ef499d0776fb16b7ab87,38,0,0,HS-grad,9,215646,40,<=50K,Divorced,United-States,Handlers-cleaners,White,Not-in-family,Male,Private
3,5d21ef499d0776fb16b7ab88,53,0,0,11th,7,234721,40,<=50K,Married-civ-spouse,United-States,Handlers-cleaners,Black,Husband,Male,Private
4,5d21ef499d0776fb16b7ab89,28,0,0,Bachelors,13,338409,40,<=50K,Married-civ-spouse,Cuba,Prof-specialty,Black,Wife,Female,Private


In [20]:
income_df['education'].describe()

count        65122
unique          16
top        HS-grad
freq         21002
Name: education, dtype: object